# 感情分析

## 映画レビューデータセット

In [1]:
import pyprind
import pandas as pd
import os
import sys

# データセットを作る
basepath = './aclImdb'

labels = {'pos': 1, 'neg': 0}

# 50,000件のレビューを処理する間の進捗状況を表示するバーを作る。プログレスバー。
pbar = pyprind.ProgBar(50000, stream=sys.stdout)
data = []
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
            data.append([txt, labels[l]])  # ← append to list
            pbar.update()

df = pd.DataFrame(data, columns=['review', 'sentiment'])  # ← 最後にDataFrame化


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:07


In [2]:
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
# データセットを保存する
df.to_csv('movie_data.csv', encoding='utf-8', index=False)

In [3]:
df = pd.read_csv('./movie_data.csv')
df.head(3)

,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0


In [4]:
df.shape

(50000, 2)

## Bowモデルの導入

1. 一意なトークンからなる語彙を作成する。
2. 各文書での各単語の出現回数を含んだ特徴量ベクトルを構築する。  

特徴量ベクトルは疎になる。

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array(
    ['The sun is shining', 'The weather is sweet', 'The sun is shining and the weather is sweet, and one and one is two.']
)
bag = count.fit_transform(docs)
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [6]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [3 3 2 1 1 1 2 1 1]]


### TF-IDF
TF（単語の頻出頻度）とIDF（逆文書頻度）の積として定義される。  
この手法を利用すれば、特徴量ベクトルに頻繁に出現する単語の重みを減らすことができる。

In [7]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.66 0.39 0.44 0.17 0.17 0.17 0.26 0.22 0.17]]


### クリーニング
Bowする前に不要な文字をすベて取り除く。  
例ではreライブラリを使って正規表現で処理している。

In [8]:
df.loc[0, 'review'][-50:]

'to Star Cinema!! Way to go, Jericho and Claudine!!'

In [9]:
import re

# 句読点を取り除き、絵文字を抽出して置き換える前処理関数を定義
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

<>:6: SyntaxWarning: invalid escape sequence '\)'
<>:7: SyntaxWarning: invalid escape sequence '\W'
<>:6: SyntaxWarning: invalid escape sequence '\)'
<>:7: SyntaxWarning: invalid escape sequence '\W'
/var/folders/zg/773ptkr55z99zw26dvy19_v00000gn/T/ipykernel_26086/516108431.py:6: SyntaxWarning: invalid escape sequence '\)'
  emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
/var/folders/zg/773ptkr55z99zw26dvy19_v00000gn/T/ipykernel_26086/516108431.py:7: SyntaxWarning: invalid escape sequence '\W'
  text = re.sub('[\W]+', ' ', text.lower()) +\


In [10]:
preprocessor(df.loc[0, 'review'][-50:])
preprocessor("</a>This :) is :( a test :-)!")

'this is a test :) :( :)'

In [11]:
df['review'] = df['review'].apply(preprocessor)

### 文書をトークン化

In [12]:
# 文書をトークン化
def tokenizer(text):
    return text.split()

tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [13]:
# ワードをステミング。関連する単語を同じ語幹にマッピングする。
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [14]:
# 127個のストップワードをダウンロードする
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nagairyousuke/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
from nltk.corpus import stopwords

# ストップワードを除外する
stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:]
if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

## 文書を分類-ロジスティック回帰モデル

In [16]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

# CountVectorizerと
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None,
                        token_pattern=None)

# 'ステップ名__パラメータ名'の形式でパラメータを指定する
param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l2'],
               'clf__C': [1.0, 10.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l2'],
               'clf__C': [1.0, 10.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(solver='liblinear'))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=2, # ログの出力レベル
                           n_jobs=-1)

gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x105f2c360>; total time=   2.8s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x1057bc360>; total time=   3.0s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x102f5c360>; total time=   3.0s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x102750360>; total time=   3.1s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x102368360>; total time=   3.1s
[CV] END clf__C=10.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x104abc360>; tota

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False,
                                                        token_pattern=None)),
                                       ('clf',
                                        LogisticRegression(solver='liblinear'))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [1.0, 10.0], 'clf__penalty': ['l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [None],
                          'vect__tokenizer': [<function tokenizer at 0x160929bc0>,
                                              <function tokenizer_porter...
                          'vect__stop_words': [['a', 'about', 'above', 'after',
                                                'again', 'against', 'ain',
                                                'all', 'am', 'an', 'and', 'any',
                                                'are', 'aren', "aren't", 'as',
                                                'at', 'be', 'because', 'been',
                                                'before', 'being', 'below',
                                                'between', 'both', 'but', 'by',
                                                'can', 'couldn', "couldn't", ...],
                                               None],
                          'vect__tokenizer': [<function tokenizer at 0x160929bc0>],
                          'vect__use_idf': [False]}],
             scoring='accuracy', verbose=2)

In [20]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)

Best parameter set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x160929bc0>} 


In [ ]:
print(f'CV accuracy: {gs_lr_tfidf.best_score_:.3f}')
clf = gs_lr_tfidf.best_estimator_
print(f'Test accuracy: {clf.score(X_test, y_test):.3f}')

# 9割でレビューがポジティブかネガティブかを予測できる!

CV accuracy: 0.893
Test accuracy: 0.900
